In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
 %reload_ext autoreload

In [3]:
import os

data_dir = '/Users/username/PycharmProjects/llm-zoomcamp/cohorts/2024/04-monitoring'

os.listdir(data_dir)

['adzhumurat_hw_solution_05.ipynb',
 'homework.md',
 'utils.py',
 'adzhumurat_hw4_solution.md',
 '.ipynb_checkpoints']

In [7]:
from utils import get_ground_truth

In [59]:

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '04-monitoring/data'
filename = 'results-gpt4o-mini.csv'
ground_truth_url = f'{base_url}/{relative_url}/{filename}?raw=1'

res = get_ground_truth(ground_truth_url, os.path.join(data_dir, filename))[:300]
res[0]

{'answer_llm': 'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).',
 'answer_orig': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'document': '0227b872',
 'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp'}

In [21]:
from utils import get_or_create_embedder

model_name = 'multi-qa-mpnet-base-dot-v1'
embedder = get_or_create_embedder(data_dir, model_name)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
huggingface model loading...


/Users/username/.pyenv/versions/3.10.13/envs/llmops-env/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` instead.
  warnings.warn(warning_message, FutureWarning)


.gitattributes:   0%|          | 0.00/791 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

...

train_script.py:   0%|          | 0.00/13.9k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.2.2. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





model loadind done


# Q1

In [60]:
answer_llm = res[0]['answer_llm']
corpus_texts = [
    answer_llm
]
passage_embeddings = embedder.encode(corpus_texts, show_progress_bar=False)

print('Q1: first coord: %.3f' % passage_embeddings[0][0])

Q1: first coord: -0.422


# Q2

In [61]:
answers_llm = [i['answer_llm'] for i in res]
answers_orig = [i['answer_orig'] for i in res]
print(len(answers_llm), len(answers_orig))

300 300


In [63]:
answers_llm_embeds = embedder.encode(answers_llm, show_progress_bar=True)
answers_orig_embeds = embedder.encode(answers_orig, show_progress_bar=True)
print(answers_llm_embeds.shape, answers_orig_embeds.shape)

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

(300, 768) (300, 768)


In [37]:
import numpy as np

def get_scores(A, B):
    scores_ = np.array([np.dot(A[i], B[i]) for i in range(A.shape[0])])
    return scores_

scores = get_scores(answers_llm_embeds, answers_orig_embeds)
print(scores.shape, np.percentile(scores, 75))

(1830,) 32.38980007171631


# Q3

In [65]:
def normed_vector(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm
    return v_norm

def normalize_embeds(embeds_input):
    embeds_normed = np.vstack([normed_vector(embeds_input[i]) for i in range(embeds_input.shape[0])])
    return embeds_normed

answers_llm_embeds_normed = normalize_embeds(answers_llm_embeds)
answers_orig_embeds_normed = normalize_embeds(answers_orig_embeds)

scores_normed = get_scores(answers_llm_embeds_normed, answers_orig_embeds_normed)
print('Num scores %d, q75: %.4f' % (scores_normed.shape[0], np.percentile(scores_normed, 75)))

Num scores 300, q75: 0.8362


# Q4

In [66]:
from rouge import Rouge
rouge_scorer = Rouge()

r = res[10]

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

print('Q4: Best F-score %.3f'% scores['rouge-1']['f'])

print(r)
scores

Q4: Best F-score 0.455
{'answer_llm': "Yes, all sessions are recorded, so if you miss one, you won't miss anything. You can catch up on the content later. Additionally, you can submit your questions in advance for office hours, and those sessions are also recorded.", 'answer_orig': 'Everything is recorded, so you won’t miss anything. You will be able to ask your questions for office hours in advance and we will cover them during the live stream. Also, you can always ask questions in Slack.', 'document': '5170565b', 'question': 'Are sessions recorded if I miss one?', 'course': 'machine-learning-zoomcamp'}


{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

# Q5

In [67]:
def eval_average_score(scores, metric='f', score_list = None):
    if score_list is None:
        score_list = scores.keys()
    res = np.mean([scores[k][metric] for k in scores if k in score_list])
    return res
eval_average_score(scores)

0.35490034990035496

# Q6

In [68]:
scores_all_points = [
    eval_average_score(rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0])
    for r in res
]
print(np.mean(scores_all_points))

0.313205367339838


In [72]:
import pandas as pd

res_df = pd.DataFrame(res)
res_df.head(1)

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp


In [75]:
res_df.iloc[0].to_dict()

{'answer_llm': 'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).',
 'answer_orig': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'document': '0227b872',
 'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp'}

In [81]:
dataset_scores_array = res_df[['answer_llm', 'answer_orig']].apply(lambda x: rouge_scorer.get_scores(x.iloc[0], x.iloc[1])[0], axis=1).values

dataset_scores_array

array([{'rouge-1': {'r': 0.061224489795918366, 'p': 0.21428571428571427, 'f': 0.09523809178130524}, 'rouge-2': {'r': 0.017543859649122806, 'p': 0.07142857142857142, 'f': 0.028169010918468917}, 'rouge-l': {'r': 0.061224489795918366, 'p': 0.21428571428571427, 'f': 0.09523809178130524}},
       {'rouge-1': {'r': 0.08163265306122448, 'p': 0.26666666666666666, 'f': 0.12499999641113292}, 'rouge-2': {'r': 0.03508771929824561, 'p': 0.13333333333333333, 'f': 0.05555555225694465}, 'rouge-l': {'r': 0.061224489795918366, 'p': 0.2, 'f': 0.09374999641113295}},
       {'rouge-1': {'r': 0.32653061224489793, 'p': 0.5714285714285714, 'f': 0.41558441095631643}, 'rouge-2': {'r': 0.14035087719298245, 'p': 0.24242424242424243, 'f': 0.17777777313333343}, 'rouge-l': {'r': 0.30612244897959184, 'p': 0.5357142857142857, 'f': 0.3896103849822905}},
       {'rouge-1': {'r': 0.16326530612244897, 'p': 0.32, 'f': 0.2162162117421476}, 'rouge-2': {'r': 0.03508771929824561, 'p': 0.07142857142857142, 'f': 0.04705881911141

Creating DataFrame

In [85]:
def prepare_df_entry(scores):
    rouge_1 = scores['rouge-1']['f']
    rouge_2 = scores['rouge-2']['f']
    rouge_l = scores['rouge-l']['f']
    rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3

    return (rouge_1, rouge_2, rouge_l, rouge_avg)

scores_df = pd.DataFrame([prepare_df_entry(i) for i in dataset_scores_array], columns=['rouge_1', 'rouge_2', 'rouge_l', 'rouge_avg'])

scores_df.head()

,rouge_1,rouge_2,rouge_l,rouge_avg
0,0.095238,0.028169,0.095238,0.072882
1,0.125000,0.055556,0.093750,0.091435
2,0.415584,0.177778,0.389610,0.327658
3,0.216216,0.047059,0.189189,0.150821
4,0.142076,0.033898,0.120219,0.098731


In [57]:
all_scores = [eval_average_score(i) for i in rouge_scorer.get_scores([r['answer_llm'] for r in res], [r['answer_orig'] for r in res])]

In [87]:
scores_df['rouge_2'].mean()


0.20696501983423318